# College Recommender for students

## importing the data from Dept of Educ 

In [143]:
#all of the imports necessary to run the model
#https://www.geeksforgeeks.org/standardscaler-minmaxscaler-and-robustscaler-techniques-ml/


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer

from sklearn.metrics.pairwise import cosine_similarity

import pickle


In [144]:
college_df = pd.read_csv('2018_college_data.csv',delimiter =',')
#drop Unnamed:- super irrelevant
college_df.drop('Unnamed: 29',axis=1,inplace=True)

## removing columns/rows that provide marginal value

In [145]:
#want colleges with at least 2/3 of the cols filled out
college_df.dropna(thresh=20,inplace=True)

In [146]:
#only take public or private four year
four_year_df = college_df[(college_df['Sector of institution (HD2018)']==1) | (college_df['Sector of institution (HD2018)']==2)]

In [147]:
four_year_df['Sector of institution (HD2018)'].value_counts()
#"Sector of institution (HD2018)","1","Public, 4-year or above"
# "Sector of institution (HD2018)","2","Private not-for-profit, 4-year or above"

2.0    1297
1.0     712
Name: Sector of institution (HD2018), dtype: int64

In [148]:
sum(four_year_df['Percent admitted - total (DRVADM2018_RV)'].isna())

367

In [149]:
#drop any row that does not have any data for percent admitted because I question any school that doesn't provide
four_year_df= four_year_df[~four_year_df['Percent admitted - total (DRVADM2018_RV)'].isna()]


In [150]:
#Dropping classification of -2 since these schools are not accredited 
# "Carnegie Classification 2018: Undergraduate Profile (HD2018)","-2","Not applicable, not in Carnegie universe (not accredited or nondegree-granting)"

four_year_df = four_year_df[~(four_year_df["Carnegie Classification 2018: Undergraduate Instructional Program (HD2018)"]==-2.0)]

In [151]:
#this classification is not as helpful as originally thought
four_year_df.drop('Carnegie Classification 2018: Basic (HD2018)',axis=1,inplace=True)

In [152]:
four_year_df.rename(columns={"instnm": "School","City location of institution (HD2018)":"City","State abbreviation (HD2018)":"State","Admissions office web address (HD2018)":"Admissions Web Address","Percent admitted - total (DRVADM2018_RV)": "Percent Admitted"},inplace=True)

In [153]:
four_year_df.rename(columns={"Institution size category (HD2018)": "Institution Size Cat"},inplace=True) 

In [154]:
four_year_df.rename(columns={"Average net price-students awarded grant or scholarship aid  2018-19 (SFA1819)": "Average Net Price","Percent of undergraduate enrollment that are women (DRVEF2018_RV)":"Percent Women"},inplace=True) 

In [155]:
four_year_df.rename(columns={"SAT Math 25th percentile score (ADM2018_RV)":"SAT Math 25th","SAT Evidence-Based Reading and Writing 25th percentile score (ADM2018_RV)":"SAT Verbal 25th"},inplace=True) 

In [156]:
#reset index so I can easily ref by iloc
four_year_df.reset_index(drop=True, inplace= True)

In [157]:
#add https:// to links
four_year_df['Admissions Web Address'] = ["https://"+row if row.lstrip().startswith("w") else row for row in four_year_df['Admissions Web Address']]


In [159]:
#add https:// to links that start with a, c or m
four_year_df['Admissions Web Address'] = ["https://"+row if row.lstrip().startswith("a" or "c" or "m") else row for row in four_year_df['Admissions Web Address']]

In [160]:
school_df = four_year_df[['State','City','School',]].sort_values(by=['State','City'])

In [162]:
#1624 schools; df for user to scroll through on school list tab
school_df

,State,City,School
9,AK,Anchorage,Alaska Pacific University
1289,AK,Anchorage,University of Alaska Anchorage
1290,AK,Fairbanks,University of Alaska Fairbanks
1291,AK,Juneau,University of Alaska Southeast
53,AL,Auburn,Auburn University
...,...,...,...
1073,WV,Shepherdstown,Shepherd University
885,WV,Vienna,Ohio Valley University
1535,WV,West Liberty,West Liberty University
1557,WV,Wheeling,Wheeling University


In [163]:
type(school_df.iloc[:,2].tolist())

list

In [164]:
four_year_df

,School,Average Net Price,Percent full-time first-time receiving an award - 6 years (DRVOM2019),State,City,ZIP code (HD2018),Admissions Web Address,Longitude location of institution (HD2018),Latitude location of institution (HD2018),Sector of institution (HD2018),...,Total price for in-state students living on campus 2018-19 (DRVIC2018),Percent Admitted,Admissions yield - total (DRVADM2018_RV),Percent of undergraduate enrollment that are White (DRVEF2018_RV),Percent Women,Graduation rate total cohort (DRVGR2018_RV),Completion of college-preparatory program (ADM2018_RV),Recommendations (ADM2018_RV),SAT Math 25th,SAT Verbal 25th
0,Abilene Christian University,27883.0,62.0,TX,Abilene,79699,https://www.acu.edu/admissions,-99.709797,32.468943,2.0,...,49722.0,57.0,14.0,64.0,60.0,67.0,2.0,3.0,510.0,530.0
1,Abraham Baldwin Agricultural College,NaN,24.0,GA,Tifton,31793-2601,https://www.abac.edu/future-students/admissions,-83.528281,31.481889,1.0,...,15345.0,69.0,62.0,79.0,57.0,22.0,1.0,3.0,460.0,470.0
2,Adams State University,NaN,38.0,CO,Alamosa,81101,https://www.adams.edu/admissions/,-105.879642,37.473636,1.0,...,22708.0,99.0,27.0,42.0,44.0,27.0,2.0,3.0,430.0,440.0
3,Adelphi University,28130.0,68.0,NY,Garden City,11530-0701,https://admissions.adelphi.edu/,-73.653321,40.721439,2.0,...,58430.0,74.0,13.0,50.0,68.0,70.0,2.0,1.0,540.0,540.0
4,Adrian College,24520.0,52.0,MI,Adrian,49221-2575,https://adrian.edu/admissions/,-84.061481,41.898591,2.0,...,53481.0,65.0,19.0,67.0,48.0,45.0,2.0,3.0,470.0,470.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,Yeshivath Zichron Moshe,12857.0,6.0,NY,South Fallsburg,12779,https://yeshivathzichronmoshe.com,-74.621823,41.730480,2.0,...,21600.0,48.0,83.0,89.0,0.0,53.0,1.0,1.0,NaN,NaN
1620,York College,16748.0,52.0,NE,York,68467-2699,,-97.579352,40.869526,2.0,...,29580.0,50.0,48.0,57.0,48.0,51.0,5.0,3.0,480.0,470.0
1621,York College of Pennsylvania,21396.0,57.0,PA,York,17403-3651,https://www.ycp.edu/admissions,-76.727984,39.946140,2.0,...,34550.0,70.0,25.0,79.0,54.0,59.0,2.0,2.0,510.0,510.0
1622,Young Harris College,18431.0,33.0,GA,Young Harris,30582,https://www.yhc.edu/admissions,-83.845870,34.933523,2.0,...,43389.0,63.0,33.0,70.0,55.0,41.0,2.0,3.0,470.0,460.0


In [165]:
#four year list to ensure input is an sctual school
four_yr_lst = four_year_df['School'].tolist()

In [166]:
four_yr_lst[::-1]

['Youngstown State University',
 'Young Harris College',
 'York College of Pennsylvania',
 'York College',
 'Yeshivath Zichron Moshe',
 'Yeshivath Viznitz',
 'Yeshivath Beth Moshe',
 'Yeshivas Novominsk',
 "Yeshivas Be'er Yitzchok",
 'Yeshiva Zichron Aryeh',
 'Yeshiva Yesodei Hatorah',
 'Yeshiva University',
 'Yeshiva Toras Chaim',
 'Yeshiva Sholom Shachna',
 'Yeshiva Shaar Hatorah',
 'Yeshiva Ohr Yisrael',
 'Yeshiva Ohr Naftoli',
 'Yeshiva Ohr Elchonon Chabad West Coast Talmudical Seminary',
 'Yeshiva of the Telshe Alumni',
 'Yeshiva of Machzikai Hadas',
 'Yeshiva of Far Rockaway Derech Ayson Rabbinical Seminary',
 'Yeshiva Kollel Tifereth Elizer',
 'Yeshiva Karlin Stolin',
 'Yeshiva Gedolah Zichron Leyma',
 'Yeshiva Gedolah Shaarei Shmuel',
 'Yeshiva Gedolah of Greater Detroit',
 'Yeshiva Gedolah Kesser Torah',
 'Yeshiva Derech Chaim',
 "Yeshiva D'monsey Rabbinical College",
 'Yeshiva Bais Aharon',
 'Yale University',
 'Xavier University of Louisiana',
 'Xavier University',
 'Wright 

In [167]:
#explanation of Carnegie Mellon
for i, col in enumerate(four_year_df.columns):
    print(i,col)


0 School
1 Average Net Price
2 Percent full-time first-time receiving an award - 6 years (DRVOM2019)
3 State
4 City
5 ZIP code (HD2018)
6 Admissions Web Address
7 Longitude location of institution (HD2018)
8 Latitude location of institution (HD2018)
9 Sector of institution (HD2018)
10 Historically Black College or University (HD2018)
11 Carnegie Classification 2018: Undergraduate Instructional Program (HD2018)
12 Carnegie Classification 2018: Undergraduate Profile (HD2018)
13 Institution Size Cat
14 ROTC (IC2018_RV)
15 Study abroad (IC2018_RV)
16 Member of National Collegiate Athletic Association (NCAA) (IC2018)
17 Total price for out-of-state students living on campus 2018-19 (DRVIC2018)
18 Total price for in-state students living on campus 2018-19 (DRVIC2018)
19 Percent Admitted
20 Admissions yield - total (DRVADM2018_RV)
21 Percent of undergraduate enrollment that are White (DRVEF2018_RV)
22 Percent Women
23 Graduation rate  total cohort (DRVGR2018_RV)
24 Completion of college-prepa

## Deciding the parameters to compare on and then formatting them

In [81]:
#removed sector of institution 
four_year_num= four_year_df.iloc[:,[1,13,19,22,26,27]]

In [82]:
def whole(x):
    return "{:.0f}".format(x)

In [83]:
def currency(x):
    return "${:.0f}".format(x)

In [84]:
def percentage(x):
    return "{:.0%}".format(x/100)

In [86]:
di = {1.0: "< 1,000", 2.0: "1,000 to 4,999", 3.0: "5,000 to 9,999",4.0:"10,000 to 19,999",5.0:">=20,000"}

In [135]:
#this function is to standardized the distances and fill NaNs
def nas_with_knn(df):
    '''Takes in a df, scales it, and then returns a df with imputed values via KNN'''
    scaler = RobustScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
    scaler = MinMaxScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
    imputer = KNNImputer(n_neighbors=5)
    df = pd.DataFrame(imputer.fit_transform(df),columns = df.columns)
    return df

In [136]:
four_year_num_scaled = nas_with_knn(four_year_num)

In [137]:
#df used for cosine similarity analysis
four_year_num_scaled

,Average Net Price,Institution Size Cat,Percent Admitted,Percent Women,SAT Math 25th,SAT Verbal 25th
0,0.522694,0.50,0.57,0.60,0.447732,0.576613
1,0.446328,0.25,0.69,0.57,0.349112,0.455645
2,0.433333,0.25,0.99,0.44,0.289941,0.395161
3,0.527108,0.50,0.74,0.68,0.506903,0.596774
4,0.462600,0.25,0.65,0.48,0.368836,0.455645
...,...,...,...,...,...,...
1619,0.254190,0.00,0.48,0.00,0.597633,0.673387
1620,0.323720,0.00,0.50,0.48,0.388560,0.455645
1621,0.406776,0.25,0.70,0.54,0.447732,0.536290
1622,0.353794,0.25,0.63,0.55,0.368836,0.435484


In [132]:
cosine_sim = cosine_similarity(four_year_num_scaled,four_year_num_scaled)

In [133]:
#given a school, find thefive most similar school
def get_recommendations(school, cosine_sim=cosine_sim):
    '''Takes in a school, uses cosine sim
    to return a df of five similar schools
    '''
    indices = pd.Series(four_year_df.index,index=four_year_df['School']).drop_duplicates()

    # Get the index of the school that matches user inpurt
    idx = indices[school]

    # Get the pairwsie similarity scores of all schools with requested
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the schools based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar schools after dropping row 0
    sim_scores = sim_scores[:6]

    # Get the school indices/list comp
    indices = [i[0] for i in sim_scores]

    # Return the top 5 most similar schools
    return four_year_df[['School','City','State','Percent Admitted','Average Net Price','Percent Women','Institution Size Cat','Admissions Web Address']].iloc[indices].reset_index(drop=True)


In [134]:
get_recommendations('University of California-Berkeley')

,School,City,State,Percent Admitted,Average Net Price,Percent Women,Institution Size Cat,Admissions Web Address
0,University of California-Berkeley,Berkeley,CA,15.0,NaN,53.0,5.0,
1,Northeastern University,Boston,MA,19.0,36479.0,52.0,5.0,https://www.northeastern.edu/admissions/
2,University of Southern California,Los Angeles,CA,13.0,37521.0,51.0,5.0,https://www.usc.edu/admission/
3,University of California-Los Angeles,Los Angeles,CA,14.0,NaN,58.0,5.0,https://www.admissions.ucla.edu/
4,University of Michigan-Ann Arbor,Ann Arbor,MI,23.0,NaN,50.0,5.0,https://www.admissions.umich.edu
5,New York University,New York,NY,20.0,37739.0,58.0,5.0,https://www.nyu.edu/admissions.html


## applying formats to four_year_df for user friendliness

In [130]:
#making a copy
four_year_model = four_year_df.copy()

In [119]:
four_year_model['Average Net Price'] = four_year_model['Average Net Price'].apply(currency)

In [120]:
four_year_model['Percent Women'] = four_year_model['Percent Women'].apply(percentage)

In [121]:
four_year_model['Percent Admitted'] = four_year_model['Percent Admitted'].apply(percentage)

In [122]:
four_year_model.replace({'Institution Size Cat': di},inplace=True)

In [123]:
four_year_model['SAT Math 25th']

0       510.0
1       460.0
2       430.0
3       540.0
4       470.0
        ...  
1619      NaN
1620    480.0
1621    510.0
1622    470.0
1623    480.0
Name: SAT Math 25th, Length: 1624, dtype: float64

In [124]:
four_year_model['SAT Math 25th'] = four_year_model['SAT Math 25th'].apply(whole)

In [125]:
four_year_model['SAT Verbal 25th'] = four_year_model['SAT Verbal 25th'].apply(whole)

In [126]:
four_year_model.replace({'$nan': "not provided","nan": "not provided"}, inplace=True)

In [129]:
#final df to use in flask app
four_year_model

,School,Average Net Price,Percent full-time first-time receiving an award - 6 years (DRVOM2019),State,City,ZIP code (HD2018),Admissions Web Address,Longitude location of institution (HD2018),Latitude location of institution (HD2018),Sector of institution (HD2018),...,Total price for in-state students living on campus 2018-19 (DRVIC2018),Percent Admitted,Admissions yield - total (DRVADM2018_RV),Percent of undergraduate enrollment that are White (DRVEF2018_RV),Percent Women,Graduation rate total cohort (DRVGR2018_RV),Completion of college-preparatory program (ADM2018_RV),Recommendations (ADM2018_RV),SAT Math 25th,SAT Verbal 25th
0,Abilene Christian University,$27883,62.0,TX,Abilene,79699,https://www.acu.edu/admissions,-99.709797,32.468943,2.0,...,49722.0,57%,14.0,64.0,60%,67.0,2.0,3.0,510,530
1,Abraham Baldwin Agricultural College,not provided,24.0,GA,Tifton,31793-2601,https://www.abac.edu/future-students/admissions,-83.528281,31.481889,1.0,...,15345.0,69%,62.0,79.0,57%,22.0,1.0,3.0,460,470
2,Adams State University,not provided,38.0,CO,Alamosa,81101,https://www.adams.edu/admissions/,-105.879642,37.473636,1.0,...,22708.0,99%,27.0,42.0,44%,27.0,2.0,3.0,430,440
3,Adelphi University,$28130,68.0,NY,Garden City,11530-0701,https://admissions.adelphi.edu/,-73.653321,40.721439,2.0,...,58430.0,74%,13.0,50.0,68%,70.0,2.0,1.0,540,540
4,Adrian College,$24520,52.0,MI,Adrian,49221-2575,https://adrian.edu/admissions/,-84.061481,41.898591,2.0,...,53481.0,65%,19.0,67.0,48%,45.0,2.0,3.0,470,470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,Yeshivath Zichron Moshe,$12857,6.0,NY,South Fallsburg,12779,https://yeshivathzichronmoshe.com,-74.621823,41.730480,2.0,...,21600.0,48%,83.0,89.0,0%,53.0,1.0,1.0,not provided,not provided
1620,York College,$16748,52.0,NE,York,68467-2699,,-97.579352,40.869526,2.0,...,29580.0,50%,48.0,57.0,48%,51.0,5.0,3.0,480,470
1621,York College of Pennsylvania,$21396,57.0,PA,York,17403-3651,https://www.ycp.edu/admissions,-76.727984,39.946140,2.0,...,34550.0,70%,25.0,79.0,54%,59.0,2.0,2.0,510,510
1622,Young Harris College,$18431,33.0,GA,Young Harris,30582,https://www.yhc.edu/admissions,-83.845870,34.933523,2.0,...,43389.0,63%,33.0,70.0,55%,41.0,2.0,3.0,470,460


## Pickle files for flask app

In [128]:
#save four year dataframe
four_year_model.to_pickle("./data/four_year_df.pkl")


In [105]:
#save scaled dataframe    
four_year_num_scaled.to_pickle("./data/num_scaled_df.pkl")



In [106]:
#save school dataframe
school_df.to_pickle("./data/school_df.pkl")


## Practice for weighted df and iterrows

In [107]:
def weighted_df(w1,w2,w3,w4,w5,w6):
    weighted_df=four_year_num_scaled.copy()
    weighted_df['Percent Admitted']=weighted_df['Percent Admitted']*w1
    weighted_df['Average Net Price']=weighted_df['Average Net Price']*w2
    weighted_df['Percent Women']=weighted_df['Percent Women']*w3
    weighted_df['Institution Size Cat']=weighted_df['Institution Size Cat']*w4
    weighted_df['SAT Math 25th']=(weighted_df['SAT Math 25th']*w5)
    weighted_df['SAT Verbal 25th']=(weighted_df['SAT Verbal 25th']*w6)
    return weighted_df

In [108]:
web_df = df = weighted_df(5,10,5,15,10,1)

In [109]:
weighted_cosine = cosine_similarity(web_df,web_df)

In [110]:
aug_six = get_recommendations('Stanford University',weighted_cosine)

In [111]:
for _, row in aug_six.iloc[1:6].iterrows():
    for i in range(len(row)):
        if i <7:
            print(row[i])
        else:
            print(row[i],"test")

Yale University
New Haven
CT
6.0
18073.0
50.0
4.0
https://admissions.yale.edu test
Massachusetts Institute of Technology
Cambridge
MA
7.0
20465.0
46.0
4.0
mitadmissions.org/ test
Duke University
Durham
NC
9.0
24386.0
50.0
4.0
https://www.admissions.duke.edu test
Vanderbilt University
Nashville
TN
10.0
25855.0
51.0
4.0
https://www.vanderbilt.edu/Admissions/ test
Columbia University in the City of New York
New York
NY
6.0
21828.0
47.0
5.0
https://www.columbia.edu/content/admissions.html test
